# Description
- This ipynb file is used to run live trading.
- Users trade in real time based on strategies that have been verified or backtested successfully.

1. Load API Key
2. Upload to Main Server
   - Upload both the strategy file and the config file to the server.  
2-1. Upload strategy  
2-2. Upload config  

3. DELETE - Optional
4. Run Auto-Trade
   - Download the log_viewer.py file
   - Run the #run-system cell
   - Then, execute the terminal command to run log_viewer.py
5. Terminate
   - If you attempt to retry after receiving a session ID, you may get a message indicating that the session already exists.
   - In this case, execute terminate to delete the current session ID, then run run-system again to generate a new session ID.
6. Confirm Position Holding After Terminating

---
# Dashboard
- userkey : The personal userkey received via email  
- strategy_name : The name of the strategy used for trading  
- symbol, startDate, endDate are variables used in the "Indicator Results" panel below.  
- symbol : Among the currently held symbols, which one do you want to select to view the indicator results?  
- startDate, endDate : The period for calculating the indicator results (format: yyyy-mm-dd HH:MM)  

If you set only userkey and strategy_name, the "Current Position" panel will show the current asset status.

---

#### ⚠️ Important
 - If the `.env` file does not contain `USER_KEY` and `DATA_KEY`, you must register the `USER_KEY` and generate the configuration file using `"1. strategy_verify_test.ipynb"`.

---

#### - Original GitHub Link : https://github.com/NeoMatrixAI/nb-runner/tree/main/notebooks

# Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import warnings
warnings.filterwarnings('ignore')

# Load Api Key

In [ ]:
!pip install dotenv

In [ ]:
from dotenv import load_dotenv
import os

load_dotenv(dotenv_path= "/content/drive/MyDrive/NeoMatrixAI/module/.env")

USER_KEY = os.getenv("USER_KEY")
DATA_KEY = os.getenv("DATA_KEY")

print("USER_KEY:", USER_KEY if USER_KEY else "Not found")
print("DATA_KEY:", DATA_KEY if DATA_KEY else "Not found")

# health check

In [ ]:
import requests

root_url = f'https://aifapbt.fin.cloud.ainode.ai/{USER_KEY}/'
requests.get(root_url).json()

# upload to main server

## strategy

In [ ]:
strategy_name = "multi_period_momentum"

In [ ]:
# Upload strategy file
endpoint = 'upload/strategy/'
url = root_url + endpoint

base_path = "/content/drive/MyDrive/NeoMatrixAI"
file_path = os.path.join(base_path, strategy_name + ".py")

with open(file_path, "rb") as f:
    files = {"file": f}
    response = requests.post(url, files=files)

print("📂 Upload Response:", response.json())

In [ ]:
# Check strategy file upload
endpoint = 'upload/check/strategy/'
url = root_url + endpoint

params = {"strategy_name": strategy_name}

response = requests.get(url, params=params)
print(response.json()['content'])

In [ ]:
# Check all uploaded strategy files
endpoint = 'upload/check/all-files/strategy/'
url = root_url + endpoint

response = requests.get(url)
print(response.json())

## DELETE

In [ ]:
# # Delete uploaded strategy settings file
# endpoint = 'upload/delete/strategy/'
# url = root_url + endpoint

# params = {"strategy_name": strategy_name}

# # Sending a DELETE request
# response = requests.delete(url, params=params)
# print(response.json())

## config

In [ ]:
# Upload strategy file
endpoint = 'upload/config/'
url = root_url + endpoint

config_name = strategy_name + "_config"
base_path = "/content/drive/MyDrive/NeoMatrixAI/"
file_path = os.path.join(base_path, config_name + ".py")

with open(file_path, "rb") as f:
    files = {"file": f}
    response = requests.post(url, files=files)

print("📂 Upload Response:", response.json())

In [ ]:
# Check strategy file upload
endpoint = 'upload/check/config/'
url = root_url + endpoint

config_name = strategy_name + "_config"
params = {"config_name": config_name}

response = requests.get(url, params=params)
print(response.json()['content'])

In [ ]:
# Check all uploaded config files
endpoint = 'upload/check/all-files/config/'
url = root_url + endpoint

response = requests.get(url)
print(response.json())

## DELETE

In [ ]:
# # Delete uploaded strategy settings file
# endpoint = 'upload/delete/config/'
# url = root_url + endpoint

# config_name = strategy_name + "_config"
# params = {"config_name": config_name}

# # Sending a DELETE request
# response = requests.delete(url, params=params)
# print(response.json())

# Run Auto-Trade

In [ ]:
# download 'log viewer' module file
!wget https://raw.githubusercontent.com/NeoMatrixAI/nb-runner/main/modules/log_viewer.py -O /content/drive/MyDrive/NeoMatrixAI/module/log_viewer.py  # trade log module

In [ ]:
# run-system
endpoint = 'command/run-system'
url = root_url + endpoint

data = {
    "strategy_name": strategy_name,
    "method": "rebalancing"
}

response = requests.post(url, json=data)

try:
  session_id = response.json()['session_id']
  print('session id :',response.json()['session_id'])
  print('dashboard :',response.json()['dashboard_url'])
except:
  print('Error :',response.json()['message']['message'])

  session_id = response.json()['message']['session_id']
  print('session id :',session_id)

  print('dashboard :',response.json()['message']['dashboard_url'])

In [ ]:
# Just choose whether to set the "save_log" part to true or false, and Run it!
!python /content/drive/MyDrive/NeoMatrixAI/module/log_viewer.py --user_key {USER_KEY} --tradeType {tradeType} --session_id {session_id} --save_log true

# terminate

In [ ]:
# terminate process
# You must execute terminate to force liquidation of the current process.
endpoint = 'command/terminate/'
url = root_url + endpoint

params = {"session_id": f"{session_id}"}

response = requests.get(url, params=params)
print(response.json())

# Confirm position holding after Terminating

In [ ]:
import pandas as pd
#################### Don't Touch function ! #######################
def all_positions(user_key, productType, marginCoin):
    url = f"https://bitgettrader.fin.cloud.ainode.ai/{USER_KEY}/futures/position/all-positions"

    payload = {
        "user_key": USER_KEY,
        "all_positions": {
            "productType": productType,
            'marginCoin': marginCoin.upper()
        }
    }

    response = requests.post(url, json=payload)
    data = response.json()['data']
    df = pd.DataFrame(data)
    if df.empty:
        print('Everything has been liquidated')
        pass
    return df
################################################################

productType = 'usdt-futures'
marginCoin = 'usdt'
df = all_positions(USER_KEY, productType, marginCoin)
df

# Flash Close

In [ ]:
#################### Don't Touch function ! #######################
def flash_close_position(USER_KEY, symbol, productType, holdSide):

    url = f"https://bitgettrader.fin.cloud.ainode.ai/{USER_KEY}/futures/trade/close-position"

    payload = {
        "user_key": USER_KEY,
        "flash_close_position": {
            "symbol": symbol,
            "productType": productType,
            "holdSide": holdSide
        }
    }

    response = requests.post(url, json=payload)
    result = response.json()
    data = result.get('data')
    if data:
      df = pd.DataFrame([data])
      return df
    else:
      print(result.get('message'))

################################################################


symbol = 'BTCUSDT' # Just input the symbol name you want to liquidate. ex) BTCUSDT, ETHUSDT ...
productType = 'usdt-futures'
holdSide = None
flash_close_position(USER_KEY, symbol, productType, holdSide)